Attempting to preprocess the data into geometry objects using python...

(apparently we can't just put python into a SQL notebook, but we have the code in another notebook...)

'POLYGON( 5 5, 10 5, 10 10, 5 5)'


Attempting to process the data into geometry objects using SQL queries. We tested this out on a small subset of the data (one cell from one z layer).

In [ ]:
SELECT * INTO NewTable
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 1 AND cell = 1;

In [16]:
DECLARE @BuildString NVARCHAR(MAX);

SELECT @BuildString = COALESCE(@BuildString + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM [MouseIleum].[dbo].[NewTable];
SET @BuildString = 'POLYGON((' + @BuildString + '))';

SELECT @BuildString as polygon_text -- 'POLYGON((2053 51,2053 54,2053 57,2056 57,2059 57, ... 2053 51))'


DECLARE @PolygonFromPoints geometry = geometry::STPolyFromText(@BuildString, 0);
-- SELECT @PolygonFromPoints.STIsValid() as is_valid; -- False
SET @PolygonFromPoints = @PolygonFromPoints.MakeValid(); -- see figures
-- SELECT @PolygonFromPoints.STIsValid() as is_valid; -- True

SELECT * FROM [MouseIleum].[dbo].[MoleculesLayer0] as b
    WHERE @PolygonFromPoints.STIntersects(geometry::Point(b.[x_pixel], b.[y_pixel],0)) = 1; -- =1 is needed bc output is 0 or 1 instead of bool

SELECT gene, COUNT(molecule_id) as molecule_count
FROM [MouseIleum].[dbo].[MoleculesLayer0] as b
    WHERE @PolygonFromPoints.STIntersects(geometry::Point(b.[x_pixel], b.[y_pixel],0)) = 1
    GROUP BY gene

(1 row affected)

(9 rows affected)

(6 rows affected)

Total execution time: 00:00:01.422

polygon_text
"POLYGON((2053 51,2053 54,2053 57,2056 57,2059 57,2059 54,2059 51,2059 48,2056 48,2056 45,2056 42,2056 39,2056 36,2053 33,2050 30,2047 27,2044 27,2041 27,2038 27,2035 30,2032 33,2029 36,2029 39,2026 39,2023 42,2020 45,2017 48,2014 48,2011 45,2011 42,2008 39,2008 36,2008 33,2008 30,2008 27,2008 24,2008 21,2008 18,2008 15,2008 12,2008 9,2011 9,2014 6,2014 3,2017 3,2020 3,2023 3,2026 3,2029 3,2032 3,2035 3,2038 3,2041 3,2044 3,2047 3,2050 3,2053 3,2056 3,2059 3,2062 3,2065 3,2068 3,2071 3,2074 3,2077 3,2080 3,2083 3,2086 3,2089 3,2092 3,2095 3,2098 3,2101 3,2104 3,2107 3,2110 3,2113 3,2116 3,2119 3,2119 6,2119 9,2119 12,2116 15,2116 18,2113 18,2110 21,2107 21,2104 24,2101 24,2098 27,2095 30,2092 33,2089 36,2086 39,2083 39,2080 39,2077 39,2074 39,2071 42,2068 42,2065 42,2062 45,2053 51))"


molecule_id,gene,x_pixel,y_pixel,z_pixel,x_um,y_um,z_um,area,total_magnitude,brightness,qc_score
395858,Hmgb2,2035,25,0,-2899.468,-1354.244,2.5,6,262.3245,1.640687603,0.938004026
397030,Adgrf5,2090,15,0,-2893.401,-1355.343,2.5,7,268.5072,1.583857896,0.95613949
397218,Neat1,2046,9,0,-2898.244,-1356.03,2.5,6,348.0166,1.763448709,0.962829876
397219,Neat1,2060,19,0,-2896.668,-1354.93,2.5,9,854.578,1.977509199,0.996381418
397497,Vim,2076,21,0,-2894.926,-1354.677,2.5,11,530.3595,1.683177667,0.99683536
397498,Vim,2104,13,0,-2891.927,-1355.609,2.5,5,504.2891,2.003709577,0.977372624
397589,Acta2,2100,23,0,-2892.338,-1354.471,2.5,9,336.8791,1.573231559,0.985504357
397590,Acta2,2106,22,0,-2891.654,-1354.664,2.5,12,486.5523,1.607948284,0.996206675
398778,Fzd7,2103,16,0,-2891.983,-1355.327,2.5,5,427.7183,1.932187828,0.967747244


gene,molecule_count
Acta2,2
Adgrf5,1
Fzd7,1
Hmgb2,1
Neat1,2
Vim,2


This gives us 9 points or 9 molecules with 6 genes which intersect with this polygon/cell.

Now we need to do this 3865 more times for each z layer...

# **Creating Molecules Table with Point Geos**

## Using the data from a Python script

Imagining that we have a table containing string representations of the polygons, and a table containing string representations of the points.

In [34]:
USE MouseIleum;

DROP TABLE IF EXISTS MoleculesWithPoints;
CREATE TABLE MoleculesWithPoints (
    id smallint, 
    molecule_id smallint,
    gene nvarchar(8),
    x_pixel smallint,
    y_pixel smallint,
    z_pixel float,
    point_string NVARCHAR(18),
    z_pixel_int tinyint
);

BULK INSERT MoleculesWithPoints FROM '/var/data/processed_data/new_molecule.csv' WITH ( FIRSTROW = 2, FIELDTERMINATOR = ',');

ALTER TABLE MoleculesWithPoints
    DROP COLUMN id;

(1000 rows affected)

Total execution time: 00:00:00.142

In [35]:
SELECT TOP(100) * FROM MoleculesWithPoints;

(100 rows affected)

Total execution time: 00:00:00.020

molecule_id,gene,x_pixel,y_pixel,z_pixel,point_string,z_pixel_int
1,Maoa,1705,1271,0,POINT(1705 1271),0
2,Maoa,1725,1922,0,POINT(1725 1922),0
3,Maoa,1753,1863,0,POINT(1753 1863),0
4,Maoa,1760,1865,0,POINT(1760 1865),0
5,Maoa,1904,794,0,POINT(1904 794),0
6,Maoa,1915,1430,0,POINT(1915 1430),0
7,Maoa,1751,1873,13.76819064,POINT(1751 1873),1
8,Maoa,1780,1844,13.76819064,POINT(1780 1844),1
9,Maoa,1816,995,13.76819064,POINT(1816 995),1
10,Maoa,1822,1232,13.76819064,POINT(1822 1232),1


Now we want to convert the POINT column into actual points.

In [38]:
ALTER TABLE MoleculesWithPoints
ADD point geometry;

Commands completed successfully.

Total execution time: 00:00:00.034

In [41]:
UPDATE MoleculesWithPoints
    SET point = geometry::STGeomFromText(point_string, 0);

(1000 rows affected)

Total execution time: 00:00:00.330

In [43]:
SELECT TOP(100) * FROM MoleculesWithPoints;

(100 rows affected)

Total execution time: 00:00:00.523

molecule_id,gene,x_pixel,y_pixel,z_pixel,point_string,z_pixel_int,point
1,Maoa,1705,1271,0,POINT(1705 1271),0,0x00000000010C0000000000A49A400000000000DC9340
2,Maoa,1725,1922,0,POINT(1725 1922),0,0x00000000010C0000000000F49A400000000000089E40
3,Maoa,1753,1863,0,POINT(1753 1863),0,0x00000000010C0000000000649B4000000000001C9D40
4,Maoa,1760,1865,0,POINT(1760 1865),0,0x00000000010C0000000000809B400000000000249D40
5,Maoa,1904,794,0,POINT(1904 794),0,0x00000000010C0000000000C09D400000000000D08840
6,Maoa,1915,1430,0,POINT(1915 1430),0,0x00000000010C0000000000EC9D400000000000589640
7,Maoa,1751,1873,13.76819064,POINT(1751 1873),1,0x00000000010C00000000005C9B400000000000449D40
8,Maoa,1780,1844,13.76819064,POINT(1780 1844),1,0x00000000010C0000000000D09B400000000000D09C40
9,Maoa,1816,995,13.76819064,POINT(1816 995),1,0x00000000010C0000000000609C400000000000188F40
10,Maoa,1822,1232,13.76819064,POINT(1822 1232),1,0x00000000010C0000000000789C400000000000409340


In [45]:
SELECT COUNT(molecule_id) FROM MoleculesWithPoints;
SELECT COUNT(molecule_id) FROM Molecules;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.957

(No column name)
1000


(No column name)
819665


Oops! This has only 1000 rows. Let's try with the original data since that seems like it's doable, at least for the Molecules.csv.

## Using the data from the raw data files

In [52]:
USE MouseIleum;

DROP TABLE IF EXISTS MoleculesWithPoints;
SELECT molecule_id, gene, x_pixel, y_pixel, z_pixel INTO MoleculesWithPoints FROM Molecules;

(819665 rows affected)

Total execution time: 00:00:02.090

Right number of rows this time!

In [53]:
ALTER TABLE MoleculesWithPoints
    ADD z_layer tinyint,
        point geometry;

Commands completed successfully.

Total execution time: 00:00:00.026

In [54]:
UPDATE MoleculesWithPoints
    SET point = geometry::STGeomFromText('POINT(' + CONVERT(VARCHAR(5), x_pixel) + ' ' + CONVERT(VARCHAR(5), y_pixel) + ')', 0)

(819665 rows affected)

Total execution time: 00:00:30.556

In [64]:
UPDATE MoleculesWithPoints
    SET z_layer = CAST(ROUND(z_pixel /13.76819064, 0) AS int);

(819665 rows affected)

Total execution time: 00:00:05.137

In [65]:
SELECT TOP(100) * FROM MoleculesWithPoints;

(100 rows affected)

Total execution time: 00:00:00.060

molecule_id,gene,x_pixel,y_pixel,z_pixel,z_layer,point
1,Maoa,1705,1271,0,0,0x00000000010C0000000000A49A400000000000DC9340
2,Maoa,1725,1922,0,0,0x00000000010C0000000000F49A400000000000089E40
3,Maoa,1753,1863,0,0,0x00000000010C0000000000649B4000000000001C9D40
4,Maoa,1760,1865,0,0,0x00000000010C0000000000809B400000000000249D40
5,Maoa,1904,794,0,0,0x00000000010C0000000000C09D400000000000D08840
6,Maoa,1915,1430,0,0,0x00000000010C0000000000EC9D400000000000589640
7,Maoa,1751,1873,13.76819064,1,0x00000000010C00000000005C9B400000000000449D40
8,Maoa,1780,1844,13.76819064,1,0x00000000010C0000000000D09B400000000000D09C40
9,Maoa,1816,995,13.76819064,1,0x00000000010C0000000000609C400000000000188F40
10,Maoa,1822,1232,13.76819064,1,0x00000000010C0000000000789C400000000000409340


This looks good! Yay points and z layers as integers.

Double checking that the z\_layer matches up with z\_pixel (fun fact: it didn't at first but now we fixed it).

In [66]:
SELECT COUNT(*), z_layer FROM MoleculesWithPoints 
GROUP BY z_layer
ORDER BY z_layer asc;

SELECT COUNT(*), z_pixel FROM MoleculesWithPoints 
GROUP BY z_pixel
ORDER BY z_pixel asc;

(9 rows affected)

(9 rows affected)

Total execution time: 00:00:00.695

(No column name),z_layer
99672,0
108756,1
113329,2
112233,3
105632,4
93646,5
77545,6
61641,7
47211,8


(No column name),z_pixel
99672,0
108756,13.76819064
113329,27.53638128
112233,41.30457192
105632,55.07276256
93646,68.8409532
77545,82.60914384
61641,96.37733447
47211,110.1455251


# Creating a table with Polygon Geos

## Using results from python script

Imagining that we have a table with just a column with polygons represented as strings

z\_index, cell, polygon\_string

In [ ]:
USE MouseIleum;

DROP TABLE IF EXISTS CellPolygons;
CREATE TABLE CellPolygons (
    id smallint, 
    z_index tinyint,
    cell smallint,
    polygon_string NVARCHAR(MAX)
);

BULK INSERT CellPolygons FROM '/var/data/processed_data/new_polygons.csv' WITH ( FIRSTROW = 2, FIELDTERMINATOR = ',');

UPDATE MoleculesWithPoints
    SET polygon = geometry::STGeomFromText(point_string, 0);


## Investigating validity of the raw polygon data

Now going back to test how multipolygons and polygon validity work.

In [67]:
USE MouseIleum;
SELECT * INTO LevelOneFirstCell
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 1 AND cell = 1;

(103 rows affected)

Total execution time: 00:00:00.262

In [68]:
DECLARE @BuildString NVARCHAR(MAX);

SELECT @BuildString = COALESCE(@BuildString + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM [MouseIleum].[dbo].[LevelOneFirstCell];
SET @BuildString = 'POLYGON((' + @BuildString + '))';

DECLARE @PolygonFromPoints geometry = geometry::STPolyFromText(@BuildString, 0);
-- SELECT @PolygonFromPoints.STIsValid() as is_valid; -- False
SET @PolygonFromPoints = @PolygonFromPoints.MakeValid(); 
    
SELECT @PolygonFromPoints.STIntersects(geometry::Point(2060, 20, 0)) -- point in the mother polygon
SELECT @PolygonFromPoints.STIntersects(geometry::Point(2056, 55, 0)) -- point in the baby polygon

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.020

(No column name)
1


(No column name)
1


Indeed intersection can work on both subpolygons. :)

In [69]:
USE MouseIleum;
SELECT * INTO LevelOneSecondCell
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 1 AND cell = 2;

(125 rows affected)

Total execution time: 00:00:00.232

In [75]:
DECLARE @BuildString NVARCHAR(MAX);

SELECT @BuildString = COALESCE(@BuildString + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM [MouseIleum].[dbo].[LevelOneSecondCell];
SET @BuildString = 'POLYGON((' + @BuildString + '))';

-- SELECT @BuildString as polygon_text -- 'POLYGON((2053 51,2053 54,2053 57,2056 57,2059 57, ... 2053 51))'


DECLARE @PolygonFromPoints geometry = geometry::STPolyFromText(@BuildString, 0);
SELECT @PolygonFromPoints.STIsValid() as is_valid; -- is it valid?
SET @PolygonFromPoints = @PolygonFromPoints.MakeValid(); --optional for polygons which are already valid
SELECT @PolygonFromPoints.STAsText();


(1 row affected)

(1 row affected)

Total execution time: 00:00:00.024

is_valid
1


(No column name)
"POLYGON ((2134 120, 2137 117, 2140 114, 2143 111, 2146 108, 2149 108, 2152 108, 2155 108, 2158 108, 2161 108, 2164 108, 2167 111, 2170 111, 2173 111, 2176 111, 2179 111, 2182 108, 2185 108, 2188 105, 2191 102, 2191 99, 2191 96, 2191 93, 2191 90, 2191 87, 2191 84, 2191 81, 2191 78, 2191 75, 2194 72, 2194 69, 2197 66, 2197 63, 2197 60, 2200 57, 2200 54, 2203 51, 2203 48, 2206 45, 2209 42, 2209 39, 2212 36, 2212 33, 2212 30, 2209 27, 2209 24, 2209 21, 2206 18, 2206 15, 2203 12, 2203 9, 2200 6, 2200 3, 2197 3, 2194 3, 2191 3, 2188 3, 2185 3, 2182 3, 2179 3, 2176 3, 2173 3, 2170 3, 2167 3, 2164 3, 2161 3, 2158 3, 2155 3, 2152 3, 2149 3, 2146 3, 2143 3, 2140 3, 2137 3, 2134 3, 2131 3, 2128 3, 2125 3, 2122 3, 2122 6, 2122 9, 2122 12, 2119 15, 2119 18, 2116 21, 2113 21, 2110 24, 2107 24, 2104 27, 2101 27, 2098 30, 2095 33, 2092 36, 2089 39, 2089 42, 2092 45, 2095 48, 2098 51, 2101 54, 2104 57, 2107 60, 2107 63, 2110 66, 2110 69, 2113 72, 2113 75, 2116 78, 2116 81, 2116 84, 2119 87, 2119 90, 2119 93, 2119 96, 2119 99, 2119 102, 2119 105, 2116 108, 2116 111, 2116 114, 2119 117, 2122 120, 2125 123, 2128 123, 2131 123, 2134 120))"


This is the naive approach.

Maybe we can try to do it a few times at once, in temporary tables (still being naive)...

In [81]:
USE MouseIleum;
DROP TABLE IF EXISTS #LevelOneThirdCell, #LevelTwoThirdCell, #LevelThreeThirdCell, #LevelFourThirdCell;
SELECT * INTO #LevelOneThirdCell
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 1 AND cell = 3;
SELECT * INTO #LevelTwoThirdCell
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 2 AND cell = 3;
SELECT * INTO #LevelThreeThirdCell
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 3 AND cell = 3;
SELECT * INTO #LevelFourThirdCell
    FROM [MouseIleum].[dbo].[Cells]
    WHERE z = 4 AND cell = 3;

DECLARE @BuildString13 NVARCHAR(MAX);

SELECT @BuildString13 = COALESCE(@BuildString13 + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM #LevelOneThirdCell;
SET @BuildString13 = 'POLYGON((' + @BuildString13 + '))';

DECLARE @PolygonFromPoints13 geometry = geometry::STPolyFromText(@BuildString13, 0);
-- SELECT @PolygonFromPoints13.STIsValid() as is_valid13; -- is it valid?
SET @PolygonFromPoints13 = @PolygonFromPoints13.MakeValid(); --optional for polygons which are already valid
SELECT @PolygonFromPoints13.STAsText();

DECLARE @BuildString23 NVARCHAR(MAX);

SELECT @BuildString23 = COALESCE(@BuildString23 + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM #LevelTwoThirdCell;
SET @BuildString23 = 'POLYGON((' + @BuildString23 + '))';

DECLARE @PolygonFromPoints23 geometry = geometry::STPolyFromText(@BuildString23, 0);
-- SELECT @PolygonFromPoints23.STIsValid() as is_valid23; -- is it valid?
SET @PolygonFromPoints23 = @PolygonFromPoints23.MakeValid(); --optional for polygons which are already valid
SELECT @PolygonFromPoints23.STAsText();

DECLARE @BuildString33 NVARCHAR(MAX);

SELECT @BuildString33 = COALESCE(@BuildString33 + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM #LevelThreeThirdCell;
SET @BuildString33 = 'POLYGON((' + @BuildString33 + '))';

-- SELECT @BuildString33;

DECLARE @PolygonFromPoints33 geometry = geometry::STPolyFromText(@BuildString33, 0);
-- SELECT @PolygonFromPoints33.STIsValid() as is_valid33; -- is it valid?
SET @PolygonFromPoints33 = @PolygonFromPoints33.MakeValid(); --optional for polygons which are already valid
SELECT @PolygonFromPoints33.STAsText();

DECLARE @BuildString43 NVARCHAR(MAX);

SELECT @BuildString43 = COALESCE(@BuildString43 + ',', '') + CAST([x] AS NVARCHAR(50)) + ' ' + CAST([y] AS NVARCHAR(50))
    FROM #LevelFourThirdCell;
SET @BuildString43 = 'POLYGON((' + @BuildString43 + '))';

-- SELECT @BuildString43;

DECLARE @PolygonFromPoints43 geometry = geometry::STPolyFromText(@BuildString43, 0);
-- SELECT @PolygonFromPoints43.STIsValid() as is_valid43; -- is it valid?
SET @PolygonFromPoints43 = @PolygonFromPoints43.MakeValid(); --optional for polygons which are already valid
SELECT @PolygonFromPoints43.STAsText();


SELECT @PolygonFromPoints13.STIsValid() as is_valid13, @PolygonFromPoints23.STIsValid() as is_valid23, @PolygonFromPoints33.STIsValid() as is_valid33, @PolygonFromPoints43.STIsValid() as is_valid43; -- are they valid?

(9 rows affected)

(111 rows affected)

(103 rows affected)

(119 rows affected)

(1 row affected)

...

(No column name)
"POLYGON ((2134 135, 2134 132, 2134 129, 2131 129, 2128 129, 2128 132, 2128 135, 2131 135, 2134 135))"


(No column name)
"POLYGON ((2104 135, 2107 135, 2110 135, 2113 135, 2116 135, 2116 132, 2116 129, 2116 126, 2119 123, 2122 123, 2125 120, 2128 117, 2128 114, 2128 111, 2128 108, 2125 105, 2125 102, 2122 99, 2122 96, 2122 93, 2119 90, 2119 87, 2119 84, 2119 81, 2116 78, 2116 75, 2113 72, 2110 69, 2107 66, 2104 63, 2101 63, 2098 60, 2098 57, 2095 54, 2095 51, 2095 48, 2095 45, 2095 42, 2092 39, 2089 39, 2086 39, 2083 42, 2080 42, 2077 45, 2074 48, 2071 48, 2068 48, 2065 51, 2062 51, 2059 51, 2056 51, 2053 54, 2050 54, 2047 57, 2044 57, 2041 57, 2038 57, 2035 57, 2032 57, 2029 60, 2026 60, 2023 60, 2020 60, 2017 60, 2014 63, 2014 66, 2011 69, 2011 72, 2011 75, 2011 78, 2011 81, 2011 84, 2011 87, 2011 90, 2011 93, 2011 96, 2011 99, 2011 102, 2014 102, 2017 102, 2020 102, 2023 105, 2026 108, 2029 111, 2029 114, 2032 117, 2035 120, 2038 123, 2038 126, 2041 126, 2044 126, 2047 126, 2050 123, 2053 123, 2056 123, 2059 123, 2062 123, 2065 123, 2068 123, 2071 123, 2074 123, 2077 126, 2080 126, 2083 126, 2086 129, 2089 129, 2092 129, 2095 132, 2098 132, 2101 132, 2104 135))"


(No column name)
"POLYGON ((2098 126, 2095 126, 2092 126, 2089 126, 2086 126, 2083 123, 2080 123, 2077 123, 2074 123, 2071 123, 2068 123, 2065 123, 2062 123, 2059 123, 2056 123, 2053 123, 2050 123, 2047 123, 2044 123, 2041 123, 2038 123, 2035 123, 2035 120, 2032 117, 2032 114, 2029 111, 2029 108, 2026 105, 2023 102, 2020 99, 2020 96, 2017 93, 2017 90, 2014 87, 2014 84, 2014 81, 2014 78, 2014 75, 2014 72, 2014 69, 2014 66, 2014 63, 2014 60, 2014 57, 2017 57, 2020 57, 2023 57, 2026 57, 2029 54, 2032 54, 2035 54, 2038 51, 2041 51, 2044 51, 2047 51, 2050 48, 2053 48, 2056 48, 2059 45, 2062 45, 2065 45, 2068 45, 2071 42, 2074 42, 2077 42, 2080 42, 2083 42, 2086 39, 2089 39, 2092 39, 2095 39, 2095 42, 2095 45, 2095 48, 2095 51, 2095 54, 2095 57, 2098 60, 2101 63, 2104 66, 2107 66, 2110 69, 2113 72, 2116 72, 2116 75, 2119 78, 2119 81, 2119 84, 2119 87, 2116 90, 2116 93, 2116 96, 2116 99, 2116 102, 2116 105, 2116 108, 2116 111, 2113 114, 2110 117, 2107 120, 2104 123, 2101 123, 2098 126))"


(No column name)
"POLYGON ((2050 138, 2047 138, 2047 135, 2044 132, 2041 132, 2038 129, 2035 129, 2032 126, 2029 123, 2026 120, 2023 117, 2020 114, 2020 111, 2017 108, 2017 105, 2017 102, 2017 99, 2017 96, 2017 93, 2017 90, 2017 87, 2017 84, 2017 81, 2017 78, 2017 75, 2017 72, 2017 69, 2017 66, 2020 66, 2023 63, 2023 60, 2026 57, 2029 54, 2029 51, 2029 48, 2032 45, 2032 42, 2035 42, 2038 42, 2041 42, 2044 42, 2047 42, 2050 42, 2053 39, 2056 39, 2059 39, 2062 39, 2065 36, 2068 33, 2071 33, 2074 30, 2077 30, 2080 30, 2083 30, 2086 33, 2089 33, 2092 33, 2095 33, 2098 30, 2101 30, 2104 27, 2107 27, 2110 27, 2113 24, 2116 27, 2116 30, 2116 33, 2116 36, 2113 39, 2113 42, 2110 42, 2107 45, 2104 48, 2104 51, 2101 54, 2101 57, 2101 60, 2104 63, 2107 66, 2110 66, 2113 69, 2116 69, 2119 72, 2122 72, 2125 75, 2125 78, 2125 81, 2125 84, 2125 87, 2122 90, 2122 93, 2119 96, 2119 99, 2119 102, 2119 105, 2116 105, 2113 108, 2110 108, 2107 111, 2104 114, 2104 117, 2101 120, 2098 123, 2095 126, 2092 129, 2089 129, 2086 132, 2083 132, 2080 132, 2077 132, 2074 132, 2071 135, 2068 135, 2065 135, 2062 135, 2059 135, 2056 135, 2053 135, 2050 138))"


is_valid13,is_valid23,is_valid33,is_valid43
1,1,1,1


So this is kinda silly, but it goes to show that of the 6 polygons we've looked at so far, only z=1, cell=1 is invalid.